In [1]:
%%capture
!pip install scikit-plot
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import validation_curve
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from scikitplot.metrics import plot_roc
from scikitplot.metrics import plot_precision_recall
from scikitplot.metrics import plot_cumulative_gain
from scikitplot.metrics import plot_lift_curve
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split, cross_val_score 

from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score
import itertools

### MONK1

import and clean train and test sets

In [2]:
#import&clean Monk_train 1
monk1_train = pd.read_csv('monks-1_train.csv', sep=' ', header= None)
monk1_train.drop([0,8], axis=1, inplace = True)
rename_dict = {}
for i in range(2,8): 
  rename_dict[i] = f"attr_{i-1}"
rename_dict.update({1:'target'})
monk1_train.rename( columns=rename_dict, inplace =True)
monk1_train = monk1_train[list(monk1_train)[1:] + list(monk1_train)[:-6]]

In [3]:
#import&clean Monk_test 1
monk1_test = pd.read_csv('monks-1_test.csv', sep=' ', header= None)
monk1_test.drop([0,8], axis=1, inplace = True)
rename_dict = {}
for i in range(2,8): 
  rename_dict[i] = f"attr_{i-1}"
rename_dict.update({1:'target'})
monk1_test.rename( columns=rename_dict, inplace =True)
monk1_test = monk1_test[list(monk1_test)[1:] + list(monk1_test)[:-6]]

#remove training set from test set
monk_temp = monk1_test.append(monk1_train, ignore_index =True)
duplicated_indexes = monk_temp.duplicated(keep=False)
monk1_test = monk_temp.drop(monk_temp.loc[duplicated_indexes,:].index).copy()

In [4]:
X = monk1_train.iloc[:,:-1]
y = monk1_train['target']
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)

In [5]:
#with Pandas get_dummies
X_encoded = pd.get_dummies(monk1_train.iloc[:,:-1].astype('str'), prefix_sep='=')

home made gridsearch

In [35]:
def wrapper(func, args):
    return func(*args)

def create_combination_df():
  global params_df, previous_best
  params_df = pd.DataFrame(data = list(wrapper(itertools.product, [list(params.values())[_] for _ in range(len(params.keys()))])), columns=list(params.keys()))
  params_df = params_df.where(pd.notnull(params_df), None)
  params_df['mean_accuracy'] = None
  print("# of initial combinations: ", len(params_df.loc[params_df.mean_accuracy.isna()]))
  previous_best = 0

def update_combination_df():
  global params_df
  params_df = pd.concat([params_df, pd.DataFrame(data = list(wrapper(itertools.product, [list(params.values())[_] for _ in range(len(params.keys()))])), columns=list(params.keys()))], axis=0, ignore_index=True)
  params_df = params_df.where(pd.notnull(params_df), None)
  print("# of new combinations: ", len(params_df.loc[params_df.mean_accuracy.isna()]))

def previous_accuracy_higher_than_new(new_best_result):
  global previous_best
  if previous_best >= new_best_result:
      print('Stopping search because no improvements','\nprevious_best = ',previous_best, '\nnew_best = ',new_best_result)
      previous_best = new_best_result 
      return True
  else:
      if previous_best != 0: print(f'Continuing search: new_best ({new_best_result}) vs previous_best ({previous_best} = {round(((new_best_result-previous_best)/previous_best)*100,3)}')
      previous_best = new_best_result
      return False 

def update_parameters(percentage_update):
    global params
    up, low = 1+1*(percentage_update/100), 1-1*(percentage_update/100)
    key_list = list(advanced_search.keys())
    for key in list(params.keys()):
        if key in key_list:
            best_param_value = params_df.sort_values(by='mean_accuracy', ascending=False).reset_index().loc[0,key]
            if advanced_search[key][1] == 'int':
                try: advanced_search[key][2] == 'min=2'
                except IndexError: min_value = 1
                else: min_value = 2
                finally: 
                        params[key] = [int(round(max(best_param_value-max(best_param_value*(percentage_update/100),1), min_value))) , int(round(max(best_param_value+max(best_param_value*(percentage_update/100),1), min_value)))]                        
                        #params[key] = [int(max(best_param_value*low, min_value)), int(max(best_param_value*up, min_value))]
            elif advanced_search[key][1] == 'float':
                params[key] = [best_param_value*low, best_param_value*up]
        else:
            best_param_value = params_df.sort_values(by='mean_accuracy', ascending=False).reset_index().loc[0,key]
            params[key] = [best_param_value]
    update_combination_df()

def get_average_performance(num_rep, min_samples_split, min_samples_leaf, max_features, n_estimators):
  mean_l = []
  for _ in range(num_rep):
    clf = RandomForestClassifier(criterion='gini', max_depth=None, min_samples_split= min_samples_split, min_samples_leaf= min_samples_leaf, max_features=max_features, 
                            n_estimators=n_estimators)
    mean_scores = cross_val_score(clf, X_encoded, y, cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=True), error_score='raise')
    #print('Accuracy: %0.4f (+/- %0.2f)' % (mean_scores.mean(), mean_scores.std() * 2))
    mean_l.append(mean_scores.mean())
  return (np.array(mean_l).mean())

def launch_search(crossval_repetitions=3):
  global params_df, temp_index
  print('# of crossval_repetitions: ', crossval_repetitions)
  temp_index = params_df.loc[params_df.mean_accuracy.isna()].index
  for i in temp_index:
    try:
      #params_df.loc[i,'mean_accuracy'] = get_average_performance(10, params_df.loc[i,'min_samples_split'], params_df.loc[i,'min_samples_leaf'], params_df.loc[i,'max_features'])
      params_df.loc[i,'mean_accuracy'] = wrapper(get_average_performance, list([crossval_repetitions]+ [params_df.loc[i,_] for _ in list(params.keys())])) 
      params_df.loc[i,'#crossval_repetitions'] = int(crossval_repetitions)
      print(dict(params_df.loc[i,[_ for _ in list(params_df) if _ != '#crossval_repetitions']]))
    except ValueError as E:
      print(f'skipped combination {i}:', E)
      pass  

In [36]:
params = {'min_samples_split': [2,4],
              'min_samples_leaf': [2],
              'max_features': [None],
              'n_estimators': [40]}
advanced_search = {'n_estimators':[30, 'int'], 'min_samples_leaf':[20,'int']}

create_combination_df()
for loop in range(10):
  print(f'loop {loop}')
  launch_search(crossval_repetitions=min(5+(loop*2),10))
  if previous_accuracy_higher_than_new(params_df.loc[temp_index,'mean_accuracy'].max()):
    break
  else:
    update_parameters(percentage_update=30-(loop*2))


# of initial combinations:  2
loop 0
# of crossval_repetitions:  5
{'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': None, 'n_estimators': 40, 'mean_accuracy': 0.9434615384615386}
{'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': None, 'n_estimators': 40, 'mean_accuracy': 0.9365384615384615}
# of new combinations:  4
loop 1
# of crossval_repetitions:  7
{'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': None, 'n_estimators': 28, 'mean_accuracy': 0.9471611721611721}
{'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': None, 'n_estimators': 52, 'mean_accuracy': 0.9558608058608058}
{'min_samples_split': 2, 'min_samples_leaf': 3, 'max_features': None, 'n_estimators': 28, 'mean_accuracy': 0.9113553113553114}
{'min_samples_split': 2, 'min_samples_leaf': 3, 'max_features': None, 'n_estimators': 52, 'mean_accuracy': 0.9164835164835166}
Continuing search: new_best (0.9558608058608058) vs previous_best (0.9434615384615386 = 1.314
# of new combi

In [37]:
best_performers_df = params_df.sort_values(by='mean_accuracy', ascending=False).reset_index().copy()
best_performers_df

,index,min_samples_split,min_samples_leaf,max_features,n_estimators,mean_accuracy,#crossval_repetitions
0,7,2,1,None,67,0.968661,9
1,6,2,1,None,37,0.957977,9
2,3,2,1,None,52,0.955861,7
3,10,2,1,None,50,0.954551,10
4,11,2,1,None,84,0.954231,10
5,13,2,2,None,84,0.947692,10
6,2,2,1,None,28,0.947161,7
7,8,2,2,None,37,0.946652,9
8,9,2,2,None,67,0.944943,9
9,0,2,2,None,40,0.943462,5


testing GridSearch best results

In [235]:
#repeating experiment n times to get an idea of variance and avr performance
r = {}
for _ in range(100):
  X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.1, stratify=y)
  clf = BaggingClassifier(DecisionTreeClassifier(min_samples_leaf=1, min_samples_split=2, max_features=0.8), n_estimators=200)
  clf = clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  y_pred_tr = clf.predict(X_train)
  r[_] = [accuracy_score(y_train, y_pred_tr), round(accuracy_score(y_test, y_pred),3)]

print([r.get(key) for key in [np.random.randint(0,100) for _ in range(10)]])
print("average of training accuracy: ", round(np.array([_[0] for _ in r.values()]).mean(), 4))
print("average of validation accuracy: ", round(np.array([_[1] for _ in r.values()]).mean(), 4))
print("std of multiple hold-out validation accuracy: ", round(np.array([_[1] for _ in r.values()]).std(), 4))

[[1.0, 0.923], [1.0, 0.923], [1.0, 1.0], [1.0, 1.0], [1.0, 1.0], [1.0, 1.0], [1.0, 0.923], [1.0, 1.0], [1.0, 1.0], [1.0, 1.0]]
average of training accuracy:  1.0
average of validation accuracy:  0.9754
std of multiple hold-out validation accuracy:  0.0473
